In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
from vit_for_tflite_compartible import PatchEncoder
import numpy as np

import time

import random
tf.random.set_seed(3407)
np.random.seed(3407)
random.seed(3407)

from sklearn.metrics import f1_score

from create_spectrogram import (
    create_spectrograms_from_audio_dataset, 
    extract_patches_from_audio_dataset
)
from helper_functions import (
    evaluate_prediction,
    get_file_size, 
    convert_bytes, 
    convert_prefetchdataset_to_numpy_arrays,
    predict_and_print_full_results,
    evaluate_time_of_prediction,
    full_int_model_predict,
    get_f1_scores_of_non_overlapping_partitions_full_int_q,
    get_f1_scores_of_bootstarping_partitions_full_int_q,
    )

2024-07-28 23:29:19.928464: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-28 23:29:20.235332: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-28 23:29:20.235389: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-28 23:29:20.312275: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-28 23:29:20.445957: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-28 23:29:20.447290: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [3]:
train_dataset = tf.keras.utils.audio_dataset_from_directory("../dataset/training", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)
test_dataset = tf.keras.utils.audio_dataset_from_directory("../dataset/testing", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)
val_dataset = tf.keras.utils.audio_dataset_from_directory("../dataset/validation", labels='inferred', sampling_rate=16000, batch_size=32, shuffle=True, seed=3407)

label_names = np.array(test_dataset.class_names)
print("Classes: ", label_names)

Found 11292 files belonging to 2 classes.


2024-07-28 23:29:26.436219: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-28 23:29:26.436831: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-07-28 23:29:26.593547: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA


2024-07-28 23:29:26.677378: W tensorflow_io/core/kernels/audio_video_mp3_kernels.cc:271] libmp3lame.so.0 or lame functions are not available


Found 1393 files belonging to 2 classes.
Found 1380 files belonging to 2 classes.
Classes:  ['non_target' 'target']


In [4]:
# train_spectrogram_ds = create_spectrograms_from_audio_dataset(train_dataset, sample_rate = 16000).cache().prefetch(tf.data.AUTOTUNE)
# test_spectrogram_ds = create_spectrograms_from_audio_dataset(test_dataset, sample_rate = 16000).cache().prefetch(tf.data.AUTOTUNE)
# val_spectrogram_ds = create_spectrograms_from_audio_dataset(val_dataset, sample_rate = 16000).cache().prefetch(tf.data.AUTOTUNE)

# x_train_np, y_train_np = convert_prefetchdataset_to_numpy_arrays(train_spectrogram_ds)
# x_val_np, y_val_np = convert_prefetchdataset_to_numpy_arrays(val_spectrogram_ds)
# x_test_np, y_test_np = convert_prefetchdataset_to_numpy_arrays(test_spectrogram_ds)

In [5]:
train_patches_ds = extract_patches_from_audio_dataset(train_dataset, sample_rate = 16000, patch_size=8).cache().prefetch(tf.data.AUTOTUNE)
test_patches_ds = extract_patches_from_audio_dataset(test_dataset, sample_rate = 16000, patch_size=8).cache().prefetch(tf.data.AUTOTUNE)
val_patches_ds = extract_patches_from_audio_dataset(val_dataset, sample_rate = 16000, patch_size=8).cache().prefetch(tf.data.AUTOTUNE)

x_train_np, y_train_np = convert_prefetchdataset_to_numpy_arrays(train_patches_ds, data_type="not spectrogram")
x_test_np, y_test_np = convert_prefetchdataset_to_numpy_arrays(test_patches_ds, data_type="not spectrogram")
x_val_np, y_val_np = convert_prefetchdataset_to_numpy_arrays(val_patches_ds, data_type="not spectrogram")

In [6]:
len(x_test_np)

1393

In [7]:
BASE_MODEL_FILE_NAME = "../spectrogram_models_from_notebooks/vit/hpo/vit_mel_spec_final.keras"
baseline_model = tf.keras.models.load_model(BASE_MODEL_FILE_NAME, compile=True, custom_objects={ "PatchEncoder": PatchEncoder})
baseline_model.summary()
convert_bytes(get_file_size(BASE_MODEL_FILE_NAME), "MB")

Model: "model_29"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_40 (InputLayer)       [(None, 23, 10, 64)]         0         []                            
                                                                                                  
 reshape_39 (Reshape)        (None, 230, 64)              0         ['input_40[0][0]']            
                                                                                                  
 dense_197 (Dense)           (None, 230, 8)               520       ['reshape_39[0][0]']          
                                                                                                  
 patch_encoder_36 (PatchEnc  (None, 230, 8)               1840      ['dense_197[0][0]']           
 oder)                                                                                     

In [8]:
print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
) = predict_and_print_full_results(baseline_model, x_test_np, y_test_np, model_format="keras")


Test dataset:
44/44 [==============================] - 2s 39ms/step
Basic assessment of the whole dataset (without any partitions):
Accuracy: 97.85%
Recall: 96.51%
Precision: 96.93%
F1-score: 96.72%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9685420863541188

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9609787298894354


### Convert to TF Lite format

In [9]:
converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
tflite_model = converter.convert()


INFO:tensorflow:Assets written to: /tmp/tmp13wces2o/assets


INFO:tensorflow:Assets written to: /tmp/tmp13wces2o/assets
2024-07-28 23:30:53.109965: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-07-28 23:30:53.110036: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-07-28 23:30:53.110988: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp13wces2o
2024-07-28 23:30:53.114458: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-07-28 23:30:53.114500: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp13wces2o
2024-07-28 23:30:53.125675: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-07-28 23:30:53.131187: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-07-28 23:30:53.277840: I tensorflow/cc/saved_model/loader.cc:217] Running initializatio

In [10]:
print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
    ) = predict_and_print_full_results(tflite_model, x_val_np, y_val_np, model_format="tf_lite")

print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
    ) = predict_and_print_full_results(tflite_model, x_test_np, y_test_np, model_format="tf_lite")

print("\nTime for Test dataset:")
evaluate_time_of_prediction(tflite_model, x_test_np, y_test_np, model_format="tf_lite")

TF_LITE_MODEL_FILE_NAME = "../spectrogram_models_from_notebooks/vit/quantized/vit_mel_spec.tflite"
open(TF_LITE_MODEL_FILE_NAME, "wb").write(tflite_model)
print("\n")
print("Model file name: ", TF_LITE_MODEL_FILE_NAME)
convert_bytes(get_file_size(TF_LITE_MODEL_FILE_NAME), "KB")

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Validation dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 97.68%
Recall: 97.03%
Precision: 96.22%
F1-score: 96.62%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.966087575828162

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9643551212009652

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 97.85%
Recall: 96.51%
Precision: 96.93%
F1-score: 96.72%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9685420863541188

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9687468466377277

Time for Test dataset:
Accuracy: 97.85%
Recall: 96.51%
Precision: 96.93%
F1-score: 96.72%

Time to make a prediction for a single data point
Mean: 0.002 seconds
Max: 0.005 seconds
M

### Dynamic range quantization

In [11]:
dynamic_range_quant_converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
dynamic_range_quant_converter.optimizations = [tf.lite.Optimize.DEFAULT]
dynamic_range_quant_model = dynamic_range_quant_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp_v8a5g7i/assets


INFO:tensorflow:Assets written to: /tmp/tmp_v8a5g7i/assets
2024-07-28 23:31:29.062042: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-07-28 23:31:29.062111: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-07-28 23:31:29.062329: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp_v8a5g7i
2024-07-28 23:31:29.069052: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-07-28 23:31:29.069171: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp_v8a5g7i
2024-07-28 23:31:29.085418: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-07-28 23:31:29.251790: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmp_v8a5g7i
2024-07-28 23:31:29.302876: I tensorflow/cc/saved_model/loader.cc:316] SavedModel

In [12]:
print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
    ) = predict_and_print_full_results(dynamic_range_quant_model, x_val_np, y_val_np, model_format="tf_lite")

print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
    ) = predict_and_print_full_results(dynamic_range_quant_model, x_test_np, y_test_np, model_format="tf_lite")

print("\nTime for Test dataset:")
evaluate_time_of_prediction(dynamic_range_quant_model, x_test_np, y_test_np, model_format="tf_lite")

DRQ_MODEL_FILE_NAME = "../spectrogram_models_from_notebooks/vit/quantized/vit_mel_spec_drq.tflite"
open(DRQ_MODEL_FILE_NAME, "wb").write(dynamic_range_quant_model)
print("\n")
print("Model file name: ", DRQ_MODEL_FILE_NAME)
convert_bytes(get_file_size(DRQ_MODEL_FILE_NAME), "KB")

Validation dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 97.68%
Recall: 97.03%
Precision: 96.22%
F1-score: 96.62%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.966087575828162

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9649249485995022

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 97.85%
Recall: 96.51%
Precision: 96.93%
F1-score: 96.72%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9685420863541188

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.969772615536503

Time for Test dataset:
Accuracy: 97.85%
Recall: 96.51%
Precision: 96.93%
F1-score: 96.72%

Time to make a prediction for a single data point
Mean: 0.002 seconds
Max: 0.006 seconds
Mi

### Float16 quantization

In [13]:
float16_quant_converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
float16_quant_converter.optimizations = [tf.lite.Optimize.DEFAULT]
float16_quant_converter.target_spec.supported_types = [tf.float16]
float16_quant_model = float16_quant_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpjqqzds05/assets


INFO:tensorflow:Assets written to: /tmp/tmpjqqzds05/assets
2024-07-28 23:32:10.930826: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-07-28 23:32:10.930894: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-07-28 23:32:10.931222: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpjqqzds05
2024-07-28 23:32:10.936494: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-07-28 23:32:10.936583: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpjqqzds05
2024-07-28 23:32:10.950237: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-07-28 23:32:11.072132: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmpjqqzds05
2024-07-28 23:32:11.115158: I tensorflow/cc/saved_model/loader.cc:316] SavedModel

In [14]:
print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
    ) = predict_and_print_full_results(float16_quant_model, x_val_np, y_val_np, model_format="tf_lite")

print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
    ) = predict_and_print_full_results(float16_quant_model, x_test_np, y_test_np, model_format="tf_lite")

print("\nTime for Test dataset:")
evaluate_time_of_prediction(float16_quant_model, x_test_np, y_test_np, model_format="tf_lite")

FLOAT16_QUANT_MODEL_FILE_NAME = "../spectrogram_models_from_notebooks/vit/quantized/vit_mel_spec_float16q.tflite"
open(FLOAT16_QUANT_MODEL_FILE_NAME, "wb").write(float16_quant_model)
print("\n")
print("Model file name: ", FLOAT16_QUANT_MODEL_FILE_NAME)
convert_bytes(get_file_size(FLOAT16_QUANT_MODEL_FILE_NAME), "KB")

Validation dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 97.68%
Recall: 97.03%
Precision: 96.22%
F1-score: 96.62%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.966087575828162

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9663487582495263

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 97.85%
Recall: 96.51%
Precision: 96.93%
F1-score: 96.72%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9685420863541188

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.963586797048674

Time for Test dataset:
Accuracy: 97.85%
Recall: 96.51%
Precision: 96.93%
F1-score: 96.72%

Time to make a prediction for a single data point
Mean: 0.003 seconds
Max: 0.327 seconds
Mi

### Float Fallback quantization

In [15]:
def representative_data_gen():
  for input_value in tf.data.Dataset.from_tensor_slices(x_val_np).batch(1).take(100):
    yield [input_value]

float_fallback_quant_converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
float_fallback_quant_converter.optimizations = [tf.lite.Optimize.DEFAULT]
float_fallback_quant_converter.representative_dataset = representative_data_gen
float_fallback_quant_model = float_fallback_quant_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp2e0ajlhg/assets


INFO:tensorflow:Assets written to: /tmp/tmp2e0ajlhg/assets
/home/polina/newname/.venv/lib/python3.9/site-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2024-07-28 23:32:46.890773: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-07-28 23:32:46.890830: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-07-28 23:32:46.891026: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp2e0ajlhg
2024-07-28 23:32:46.895152: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-07-28 23:32:46.895190: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp2e0ajlhg
2024-07-28 23:32:46.908807: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-07-28 23:32

In [16]:
print("Validation dataset:")
(
    y_pred_val, 
    non_overlap_patritions_f1_scores_val, 
    bootstrap_patritions_f1_scores_val,
    ) = predict_and_print_full_results(float_fallback_quant_model, x_val_np, y_val_np, model_format="tf_lite")

print("\nTest dataset:")
(
    y_pred_test, 
    non_overlap_patritions_f1_scores_test, 
    bootstrap_patritions_f1_scores_test,
    ) = predict_and_print_full_results(float_fallback_quant_model, x_test_np, y_test_np, model_format="tf_lite")

print("\nTime for Test dataset:")
evaluate_time_of_prediction(float_fallback_quant_model, x_test_np, y_test_np, model_format="tf_lite")

FLOAT_FALLBACK_QUANT_MODEL_FILE_NAME = "../spectrogram_models_from_notebooks/vit/quantized/vit_mel_spec_fallback_q.tflite"
open(FLOAT_FALLBACK_QUANT_MODEL_FILE_NAME, "wb").write(float_fallback_quant_model)
print("\n")
print("Model file name: ", FLOAT_FALLBACK_QUANT_MODEL_FILE_NAME)
convert_bytes(get_file_size(FLOAT_FALLBACK_QUANT_MODEL_FILE_NAME), "MB")

Validation dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 96.45%
Recall: 94.07%
Precision: 95.48%
F1-score: 94.77%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.947604368833202

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9493647241831761

Test dataset:
Basic assessment of the whole dataset (without any partitions):
Accuracy: 96.41%
Recall: 92.58%
Precision: 96.36%
F1-score: 94.43%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9469991240043372

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.9451286879779413

Time for Test dataset:
Accuracy: 96.41%
Recall: 92.58%
Precision: 96.36%
F1-score: 94.43%

Time to make a prediction for a single data point
Mean: 0.003 seconds
Max: 0.01 seconds
Mi

### Full integer quantization

In [17]:
def representative_data_gen():
  for input_value in tf.data.Dataset.from_tensor_slices(x_val_np).batch(1).take(100):
    yield [input_value]

full_int_converter = tf.lite.TFLiteConverter.from_keras_model(baseline_model)
full_int_converter.optimizations = [tf.lite.Optimize.DEFAULT]
full_int_converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
full_int_converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
full_int_converter.inference_input_type = tf.uint8
full_int_converter.inference_output_type = tf.uint8

tflite_model_full_int = full_int_converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpo_btbz4f/assets


INFO:tensorflow:Assets written to: /tmp/tmpo_btbz4f/assets
/home/polina/newname/.venv/lib/python3.9/site-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2024-07-28 23:33:52.090994: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-07-28 23:33:52.091063: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-07-28 23:33:52.091249: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpo_btbz4f
2024-07-28 23:33:52.096299: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-07-28 23:33:52.096374: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpo_btbz4f
2024-07-28 23:33:52.111473: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-07-28 23:33

In [18]:
interpreter = tf.lite.Interpreter(model_content=tflite_model_full_int)
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

input:  <class 'numpy.uint8'>
output:  <class 'numpy.uint8'>


In [19]:
import pathlib

tflite_models_dir = pathlib.Path("../spectrogram_models_from_notebooks/vit/quantized/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

# Save the quantized model:
tflite_model_full_int_file = tflite_models_dir/"vit_mel_spec_full_int_q.tflite"
tflite_model_full_int_file.write_bytes(tflite_model_full_int)

62296

In [20]:
print("Validation dataset:")
predictions = full_int_model_predict(tflite_model_full_int_file, x_val_np)
evaluate_prediction(y_val_np, predictions)

print("\nDevide dataset into 10 non-overlapping patritions and get their mean F1-score")
non_overlap_patritions_f1_scores = get_f1_scores_of_non_overlapping_partitions_full_int_q(tflite_model_full_int_file, x_val_np, y_val_np)
print("Non-overlap mean F1-score: ", np.mean(non_overlap_patritions_f1_scores))

print("\nGet 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score")
bootstrap_patritions_f1_scores = get_f1_scores_of_bootstarping_partitions_full_int_q(tflite_model_full_int_file, x_val_np, y_val_np)
print("Bootstrap mean F1-score: ", np.mean(bootstrap_patritions_f1_scores))



print("\nTest dataset:")
predictions = full_int_model_predict(tflite_model_full_int_file, x_test_np)
evaluate_prediction(y_test_np, predictions)

print("\nDevide dataset into 10 non-overlapping patritions and get their mean F1-score")
non_overlap_patritions_f1_scores = get_f1_scores_of_non_overlapping_partitions_full_int_q(tflite_model_full_int_file, x_test_np, y_test_np)
print("Non-overlap mean F1-score: ", np.mean(non_overlap_patritions_f1_scores))

print("\nGet 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score")
bootstrap_patritions_f1_scores = get_f1_scores_of_bootstarping_partitions_full_int_q(tflite_model_full_int_file, x_test_np, y_test_np)
print("Bootstrap mean F1-score: ", np.mean(bootstrap_patritions_f1_scores))

print("\nTime for Test dataset:")
time_data = []
for data_point in x_test_np:
    start_time = time.time()
    predictions = full_int_model_predict(tflite_model_full_int_file, [data_point])
    elapsed_time = time.time() - start_time
    time_data.append(elapsed_time)
print("\nTime to make a prediction for a single data point")
print(f"Mean: {round(np.mean(time_data), 3)} seconds")
print(f"Max: {round(np.max(time_data), 3)} seconds")
print(f"Min: {round(np.min(time_data), 3)} seconds")

convert_bytes(get_file_size(tflite_model_full_int_file), "KB")

Validation dataset:
Accuracy: 96.67%
Recall: 94.28%
Precision: 95.91%
F1-score: 95.09%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9501546890661956

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.952361643774056

Test dataset:
Accuracy: 96.63%
Recall: 92.36%
Precision: 97.24%
F1-score: 94.74%

Devide dataset into 10 non-overlapping patritions and get their mean F1-score
Non-overlap mean F1-score:  0.9493801440675945

Get 100 bootstrap samples from dataset with 100 samples each and get their mean F1-score
Bootstrap mean F1-score:  0.951915338719718

Time for Test dataset:

Time to make a prediction for a single data point
Mean: 0.003 seconds
Max: 0.007 seconds
Min: 0.002 seconds
File size: 60.836 Kilobytes


In [21]:
interpreter = tf.lite.Interpreter(model_content=tflite_model_full_int)
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

input:  <class 'numpy.uint8'>
output:  <class 'numpy.uint8'>
